# Unita: Regularization

## Semplice classificazione basata su Logistic Regression e Tensorflow.

In [1]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle


In [2]:
# carico i dati
pickle_file = "../datasets/notMNIST.pickle"

with open(pickle_file, 'rb') as f:
  saved = pickle.load(f)
  train_dataset = saved['train_dataset']
  train_labels = saved['train_labels']
  valid_dataset = saved['valid_dataset']
  valid_labels = saved['valid_labels']
  test_dataset = saved['test_dataset']
  test_labels = saved['test_labels']
  del saved 

image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels

train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

## PROBLEMA #1

Includere la L2 regularization nella logistic regression e valutare la differenza di accuracy.

N.B. In TF il metodo nn.l2_loss(t) calcola L2 loss del tensore t. 

Sia nella logistic regression che nelle reti neurali, la L2 reg. aggiunge un fattore al loss che tiene conto della norma dei pesi, lambda è il parametro moltiplicativo.

### Alcune considerazioni operative riguardo Regression - Valutazione: 

bias <- differenza tra la funzione media e la true function; durante la sperimentazione possiamo interpretarla come una stima di quanto il ci stiamo adattando alla true function.

varianza <- misura della variabilità dei valori assunti rispetto al valor medio della funzione.


con lambda alto (alta L2 regularization) => tendiamo ad ottenere alto bias, bassa varianza.

con lambda basso (bassa L2 regularization) => tendiamo ad ottenere basso bias, alta varianza.


### Tentativi operativi per alterare bias e varianza:

eccesso bias (potenziale underfitting) -> rendere la rete più complessa, incrementare la fase di addestramento.

eccesso varianza (potenziale overfitting) -> più dati, usare la regularization (es. L2), ridurre i parametri.

In [19]:
# Rendo più veloce l'apprendimento limitando le istanze in input
train_subset = 10000

lambda_ = 0.01

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
  
    weights = tf.Variable(tf.truncated_normal([image_size * image_size, num_labels]))
    biases = tf.Variable(tf.zeros([num_labels]))
  
    logits = tf.matmul(tf_train_dataset, weights) + biases 

    # Loss function originale
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))

    # Loss function con L2 Regularization
    # La funzione nn.l2_loss restituisce un tensore corrispondente alla L2 norm dell'input
    regularizer = tf.nn.l2_loss(weights)
    # LOSS(w) + lambda * ||W||^2_2
    loss = tf.reduce_mean(loss + lambda_ * regularizer)
    
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax( tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

In [20]:
num_steps = 1801

with tf.Session(graph=graph) as session:

    tf.global_variables_initializer().run()
    print('Initialized')
    for step in range(num_steps):
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 100 == 0):
            print('Loss at step {}: {}'.format(step, l))
            print('Training accuracy: {:.1f}'.format(accuracy(predictions, 
                                                         train_labels[:train_subset, :])))
            print('Validation accuracy: {:.1f}'.format(accuracy(valid_prediction.eval(), 
                                                           valid_labels)))
    print('Test accuracy: {:.1f}'.format(accuracy(test_prediction.eval(), test_labels))) 

Initialized
Loss at step 0: 48.471702575683594
Training accuracy: 11.3
Validation accuracy: 13.1
Loss at step 100: 11.959612846374512
Training accuracy: 73.8
Validation accuracy: 73.6
Loss at step 200: 4.521285533905029
Training accuracy: 79.0
Validation accuracy: 77.8
Loss at step 300: 1.9917118549346924
Training accuracy: 82.3
Validation accuracy: 80.7
Loss at step 400: 1.1310162544250488
Training accuracy: 83.7
Validation accuracy: 82.0
Loss at step 500: 0.8350707292556763
Training accuracy: 84.2
Validation accuracy: 82.5
Loss at step 600: 0.7321088910102844
Training accuracy: 84.5
Validation accuracy: 82.8
Loss at step 700: 0.695945143699646
Training accuracy: 84.6
Validation accuracy: 82.9
Loss at step 800: 0.6831419467926025
Training accuracy: 84.6
Validation accuracy: 82.8
Loss at step 900: 0.6785733103752136
Training accuracy: 84.7
Validation accuracy: 82.8
Loss at step 1000: 0.6769261956214905
Training accuracy: 84.7
Validation accuracy: 82.9
Loss at step 1100: 0.6763243675231

### Con diversi valori di lambda otteniamo diversi valori durante l'addestramento

0.1 -> alta regolarizzazione 

	validation accuracy aumenta ma si stabilizza subito;
        
	loss iniziale alto ma si riduce e si stabilizza immediatamente


0.0001 -> bassa regolarizzazione 
 
	validation accuracy aumenta ma si stabilizza subito;

	loss iniziale basso 
	
	validation accuracy aumenta lentamente 
	
	training accuracy aumenta -> possibile overfit?
	
	rispetto a lambda=0.1 ottengo Test accuracy minore


0.01 -> ottengo la migliore Test accuracy in confront agli altri valori di lambda

## PROBLEMA #2

Includere la L2 regularization nelle reti neurali (con RELU)

In [21]:
# nodi del hidden layer
hidden_nodes= 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, hidden_nodes]))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))

    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2

    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits \
                          (labels=tf_train_labels, logits=logits_2))

    # N.B. La L2-norm viene calcolata su ogni peso della rete
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + lambda_ * regularizers)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    
    train_prediction = tf.nn.softmax(logits_2)
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    valid_prediction = tf.nn.softmax(logits_2) 
    
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    test_prediction = tf.nn.softmax(logits_2)   

In [22]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3486.230469
Minibatch accuracy: 6.2%
Validation accuracy: 23.7%
Minibatch loss at step 500: 21.257484
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 1000: 0.923752
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.750696
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.686687
Minibatch accuracy: 85.9%
Validation accuracy: 84.3%
Minibatch loss at step 2500: 0.786069
Minibatch accuracy: 78.9%
Validation accuracy: 84.2%
Minibatch loss at step 3000: 0.819205
Minibatch accuracy: 82.0%
Validation accuracy: 84.5%
Test accuracy: 90.2%


### Test Accuracy più alta; cosa succede se altero il lambda?

## PROBLEMA #3
 
Ridurre il training set a pochi batches, cosa succede?

In [24]:
num_steps = 3001

# Prendo solo le prime 500 istanze
train_dataset_2 = train_dataset[:500, :]
train_labels_2 = train_labels[:500]

with tf.Session(graph=graph) as session:
    tf.global_variables_initializer().run()
    print("Initialized")
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels_2.shape[0] - batch_size)
        batch_data = train_dataset_2[offset:(offset + batch_size), :]
        batch_labels = train_labels_2[offset:(offset + batch_size), :]

        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
        
        if (step % 500 == 0):
            print("Minibatch loss at step {}: {}".format(step, l))
            print("Minibatch accuracy: {:.1f}".format(accuracy(predictions, batch_labels)))
            print("Validation accuracy: {:.1f}".format(accuracy(valid_prediction.eval(), valid_labels)))
            
    print("Test accuracy: {:.1f}".format(accuracy(test_prediction.eval(), test_labels)))

Initialized
Minibatch loss at step 0: 3463.28564453125
Minibatch accuracy: 21.1
Validation accuracy: 36.6
Minibatch loss at step 500: 21.076845169067383
Minibatch accuracy: 100.0
Validation accuracy: 76.6
Minibatch loss at step 1000: 0.47467437386512756
Minibatch accuracy: 100.0
Validation accuracy: 78.5
Minibatch loss at step 1500: 0.29977282881736755
Minibatch accuracy: 100.0
Validation accuracy: 78.3
Minibatch loss at step 2000: 0.2808830738067627
Minibatch accuracy: 100.0
Validation accuracy: 78.3
Minibatch loss at step 2500: 0.2698928415775299
Minibatch accuracy: 100.0
Validation accuracy: 77.9
Minibatch loss at step 3000: 0.27201345562934875
Minibatch accuracy: 100.0
Validation accuracy: 78.1
Test accuracy: 84.7


### Il minibatch accuracy va subito al 100% e loss assume valori bassi.
### La validation accuracy è bassa e oscilla.
### I dati non sono sufficientemente rappresentativi oppure ci sono troppi parametri nella rete.

## PROBLEMA #4

Includere il dropout durante il training. Valutare di nuovo l'output nello scenario precedente (#3).

N.B. In TF il metodo nn.dropout() implementa il dropout.

In [25]:
num_nodes= 1024
batch_size = 128
lambda_ = 0.01

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    weights_1 = tf.Variable(tf.truncated_normal([image_size * image_size, num_nodes]))
    biases_1 = tf.Variable(tf.zeros([num_nodes]))
    weights_2 = tf.Variable(tf.truncated_normal([num_nodes, num_labels]))
    biases_2 = tf.Variable(tf.zeros([num_labels]))
    
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)

    # La probabilità di dropout (0,1] è parametrica e viene passata dal blocco session
    keep_prob = tf.placeholder(tf.float32)
    # La funzione prende in input il tensore che rappresenta gli elementi su cui valutare il dropout.
    # Con probabilità 1-keep_prob l'elemento viene scartato (cioè output corrispondente = 0).
    relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob)
    
    logits_2 = tf.matmul(relu_layer_dropout, weights_2) + biases_2

    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits \
                          (labels=tf_train_labels, logits=logits_2))
    
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2)
    loss = tf.reduce_mean(loss + lambda_ * regularizers)
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)

    train_prediction = tf.nn.softmax(logits_2)
    logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    valid_prediction = tf.nn.softmax(logits_2)
    
    logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    relu_layer= tf.nn.relu(logits_1)
    logits_2 = tf.matmul(relu_layer, weights_2) + biases_2
    test_prediction =  tf.nn.softmax(logits_2)

In [26]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    # Aggiungiamo il parametro keep_prob necessario per la Dropout regularization
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}

    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3611.326660
Minibatch accuracy: 13.3%
Validation accuracy: 37.0%
Minibatch loss at step 500: 21.366211
Minibatch accuracy: 85.2%
Validation accuracy: 84.3%
Minibatch loss at step 1000: 0.988881
Minibatch accuracy: 78.9%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.813000
Minibatch accuracy: 83.6%
Validation accuracy: 83.8%
Minibatch loss at step 2000: 0.744191
Minibatch accuracy: 85.2%
Validation accuracy: 84.0%
Minibatch loss at step 2500: 0.846502
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 3000: 0.872795
Minibatch accuracy: 80.5%
Validation accuracy: 84.0%
Test accuracy: 90.1%


### La test accuracy è identica al caso senza dropout.
### Posso impiegare meno tempo per ottenere gli stessi risultati.

## PROBLEMA #5

Prova a migliorare l'accuracy con un modello multi-layer. 
L'accuracy migliore finora ottenuta con le deep network è 97%. 

### Possibili soluzioni

Non posso aumententare i dati in ingresso

Aumento la profondità (Es. 5 hidden layers)

Riduco i parametri (Es. riducono i nodi del 50% per ogni layer più vicino all'output)

Stochastic Gradient Descent 

L2 Regularization + Drop out

Più epoche nell'apprendimento

Learning rate con decay esponenziale 

In [ ]:
import math as math

batch_size = 128
lambda_ = 0.001

# 5 hidden layers, ogni layer più vicino all'output ha un numero di layer dimezzati rispetto al precedente
hidden_nodes_1 = 1024
hidden_nodes_2 = int(hidden_nodes_1 * 0.5)
hidden_nodes_3 = int(hidden_nodes_1 * np.power(0.5, 2))
hidden_nodes_4 = int(hidden_nodes_1 * np.power(0.5, 3))
hidden_nodes_5 = int(hidden_nodes_1 * np.power(0.5, 4))

graph = tf.Graph()
with graph.as_default():

    tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Inizializzazione parametri
    
    # La distribuzione dei pesi scelti casualmente ha varianza crescente con l'aumentare della dimensione (input).
    # Invece di una stddev di default (= 1), seguiamo la Glorot initialization con RELU.
    # Glorot initialization con RELU: stddev = sqrt ( 2 / num_nodes_in_layer) 
    # Una strategia di inizializzazione utile quando si hanno architetture "dense".
    # Riferimenti: "Understanding the difficulty of training deep feedforward neural networks" Glorot, Bengio. 2010. 
    image_pixels = image_size*image_size
    weights_1 = tf.Variable(tf.truncated_normal([image_pixels, hidden_nodes_1], stddev=math.sqrt(2.0/(image_pixels))))
    biases_1 = tf.Variable(tf.zeros([hidden_nodes_1]))

    # layer 2
    weights_2 = tf.Variable(tf.truncated_normal([hidden_nodes_1, hidden_nodes_2], stddev=math.sqrt(2.0/hidden_nodes_1)))
    biases_2 = tf.Variable(tf.zeros([hidden_nodes_2]))
    
    # layer 3
    weights_3 = tf.Variable(tf.truncated_normal([hidden_nodes_2, hidden_nodes_3], stddev=math.sqrt(2.0/hidden_nodes_2)))
    biases_3 = tf.Variable(tf.zeros([hidden_nodes_3]))
    
    # layer 4
    weights_4 = tf.Variable(tf.truncated_normal([hidden_nodes_3, hidden_nodes_4], stddev=math.sqrt(2.0/hidden_nodes_3)))
    biases_4 = tf.Variable(tf.zeros([hidden_nodes_4]))
    
    # layer 5
    weights_5 = tf.Variable(tf.truncated_normal([hidden_nodes_4, hidden_nodes_5], stddev=math.sqrt(2.0/hidden_nodes_4)))
    biases_5 = tf.Variable(tf.zeros([hidden_nodes_5]))
    
    # Output layer
    weights_6 = tf.Variable(tf.truncated_normal([hidden_nodes_5, num_labels], stddev=math.sqrt(2.0/hidden_nodes_5)))
    biases_6 = tf.Variable(tf.zeros([num_labels]))
    
    
    # Architettura
    
    # layer 1 
    logits_1 = tf.matmul(tf_train_dataset, weights_1) + biases_1
    hidden_layer_1 = tf.nn.relu(logits_1)
    keep_prob = tf.placeholder(tf.float32)
    hidden_layer_1_dropout = tf.nn.dropout(hidden_layer_1, keep_prob)
    
    # layer 2
    logits_2 = tf.matmul(hidden_layer_1_dropout, weights_2) + biases_2
    hidden_layer_2 = tf.nn.relu(logits_2)
    hidden_layer_2_dropout = tf.nn.dropout(hidden_layer_2, keep_prob)
    
    # layer 3
    logits_3 = tf.matmul(hidden_layer_2_dropout, weights_3) + biases_3
    hidden_layer_3 = tf.nn.relu(logits_3)
    hidden_layer_3_dropout = tf.nn.dropout(hidden_layer_3, keep_prob)
    
    # layer 4
    logits_4 = tf.matmul(hidden_layer_3_dropout, weights_4) + biases_4
    hidden_layer_4 = tf.nn.relu(logits_4)
    hidden_layer_4_dropout = tf.nn.dropout(hidden_layer_4, keep_prob)
    
    # layer 5
    logits_5 = tf.matmul(hidden_layer_4_dropout, weights_5) + biases_5
    hidden_layer_5 = tf.nn.relu(logits_5)
    hidden_layer_5_dropout = tf.nn.dropout(hidden_layer_5, keep_prob)
    
    # Output layer
    logits_6 = tf.matmul(hidden_layer_5_dropout, weights_6) + biases_6 
    
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits \
                          (labels=tf_train_labels, logits=logits_6))

    # Considero tutti i pesi della rete
    regularizers = tf.nn.l2_loss(weights_1) + tf.nn.l2_loss(weights_2) + \
                   tf.nn.l2_loss(weights_3) + tf.nn.l2_loss(weights_4) + \
                   tf.nn.l2_loss(weights_5) + tf.nn.l2_loss(weights_6)
    loss = tf.reduce_mean(loss + lambda_ * regularizers)

    # Impiego un Time-based decay di tipo esponenziale
    
    # La funzione tf.train.exponential_decay applica un decay rate esponenziale al learning rate
    # Ha come parametri: il learning rate iniziale, l'epoca corrente (cioè i minibatch valutati finora),
    # il numero di epoche che devono trascorrere per aggiornare nuovamente il learning rate corrente,
    # e il tasso di decay (es. 0.96).
    # L'attuale learning rate viene calcolato nel seguente modo:
    # decayed_learning_rate = learning_rate * decay_rate ^ (global_step / decay_steps)

    # Alternativa: la funzione tf.train.inverse_time_decay valuta l'inverse time decay.

    # Ottengo il numero di minibatch valutati
    global_step = tf.Variable(0) 
    start_learning_rate = 0.5
    # Il decay avviene ogni 100000 steps con un decay rate di base pari a 0.96
    learning_rate = tf.train.exponential_decay(start_learning_rate, global_step, 100000, 0.96, staircase=True)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    
    
    train_prediction = tf.nn.softmax(logits_6)
    valid_logits_1 = tf.matmul(tf_valid_dataset, weights_1) + biases_1
    valid_relu_1 = tf.nn.relu(valid_logits_1)    
    valid_logits_2 = tf.matmul(valid_relu_1, weights_2) + biases_2
    valid_relu_2 = tf.nn.relu(valid_logits_2)    
    valid_logits_3 = tf.matmul(valid_relu_2, weights_3) + biases_3
    valid_relu_3 = tf.nn.relu(valid_logits_3)    
    valid_logits_4 = tf.matmul(valid_relu_3, weights_4) + biases_4
    valid_relu_4 = tf.nn.relu(valid_logits_4)    
    valid_logits_5 = tf.matmul(valid_relu_4, weights_5) + biases_5
    valid_relu_5 = tf.nn.relu(valid_logits_5)    
    valid_logits_6 = tf.matmul(valid_relu_5, weights_6) + biases_6    
    valid_prediction = tf.nn.softmax(valid_logits_6)
    
    test_logits_1 = tf.matmul(tf_test_dataset, weights_1) + biases_1
    test_relu_1 = tf.nn.relu(test_logits_1)    
    test_logits_2 = tf.matmul(test_relu_1, weights_2) + biases_2
    test_relu_2 = tf.nn.relu(test_logits_2)    
    test_logits_3 = tf.matmul(test_relu_2, weights_3) + biases_3
    test_relu_3 = tf.nn.relu(test_logits_3)    
    test_logits_4 = tf.matmul(test_relu_3, weights_4) + biases_4
    test_relu_4 = tf.nn.relu(test_logits_4)
    test_logits_5 = tf.matmul(test_relu_4, weights_5) + biases_5
    test_relu_5 = tf.nn.relu(test_logits_5)    
    test_logits_6 = tf.matmul(test_relu_5, weights_6) + biases_6    
    test_prediction = tf.nn.softmax(test_logits_6)

In [ ]:
# Incremento notevolmente il numero di epoche
num_steps = 15000
   
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
    
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
    
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))    

Initialized
Minibatch loss at step 0: 4.795702
Minibatch accuracy: 9.4%
Validation accuracy: 8.8%
Minibatch loss at step 500: 1.758690
Minibatch accuracy: 80.5%
Validation accuracy: 83.4%
Minibatch loss at step 1000: 1.520677
Minibatch accuracy: 78.1%
Validation accuracy: 84.3%
Minibatch loss at step 1500: 1.113029
Minibatch accuracy: 86.7%
Validation accuracy: 84.9%
Minibatch loss at step 2000: 0.991462
Minibatch accuracy: 86.7%
Validation accuracy: 85.3%
Minibatch loss at step 2500: 1.053444
Minibatch accuracy: 82.8%
Validation accuracy: 85.7%
Minibatch loss at step 3000: 0.944487
Minibatch accuracy: 83.6%
Validation accuracy: 85.8%
Minibatch loss at step 3500: 0.880110
Minibatch accuracy: 82.8%
Validation accuracy: 85.8%
Minibatch loss at step 4000: 1.004742
Minibatch accuracy: 79.7%
Validation accuracy: 86.2%
Minibatch loss at step 4500: 0.787686
Minibatch accuracy: 82.8%
Validation accuracy: 86.4%
Minibatch loss at step 5000: 0.692285
Minibatch accuracy: 91.4%
Validation accuracy: